In [36]:
import json
import dataset
import tweepy
import time
import datetime

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import StaleElementReferenceException

from config import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

In [126]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print("Can't authenticate")

In [182]:
user = 'seattle'

start = datetime.datetime(2017, 3, 23)
end = datetime.datetime(2017, 3, 30)

twitter_ids_filename = 'seattle_ids.json'

In [128]:
def get_twitter_auth():
    """Setup Twitter Authentication.
    
    Return: tweepy.OAuthHandler object
    """
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    return auth
    
def get_twitter_client():
    """Setup Twitter API Client.
    
    Return: tweepy.API object
    """
    auth = get_twitter_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return client

client = get_twitter_client()

In [183]:
def twitter_url(user, start, end):
    """Form url to access tweets via Twitter's search page.
    
    Return: string
    """
    # for replies to user
    #url1 = 'https://twitter.com/search?f=tweets&q=to%3A'
    # for tweets to user
    #url1 = 'https://twitter.com/search?f=tweets&q=to%40'
    # for search
    url1 = 'https://twitter.com/search?f=tweets&q='
    url2 = user + '%20since%3A' + start.strftime('%Y-%m-%d') 
    #url3 = '%20until%3A' + end.strftime('%Y-%m-%d') + '%20include%3Aretweets&src=typd'
    url3 = '%20until%3A' + end.strftime('%Y-%m-%d')
    return url1 + url2 + url3
    
def increment_day(date, i):
    """Increment day object by i days.
    
    Return: datetime object
    """
    return date + datetime.timedelta(days=i)

In [138]:
print(twitter_url(user, start, end))

https://twitter.com/search?f=tweets&q=to%40cityofseattle%20since%3A2017-03-31%20until%3A2017-03-30


In [184]:
# Adapted from https://github.com/bpb27/twitter_scraping

delay = 1  # time to wait on each page load before reading the page
driver = webdriver.Chrome() 

tweet_selector = 'li.js-stream-item'
id_selector = '.time a.tweet-timestamp'

ids = list()
for day in range((end - start).days + 1):
    # Get Twitter search url
    startDate = increment_day(start, 0)
    endDate = increment_day(start, 1)
    url = twitter_url(user, startDate, endDate)

    driver.get(url)
    time.sleep(delay)

    try:
        found_tweets = driver.find_elements_by_css_selector(tweet_selector)
        increment = 10

        # Scroll through the Twitter search page
        while len(found_tweets) >= increment:
            print('scrolling down to load more tweets')
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(delay)
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment += 10
        #print('{} tweets found, {} total'.format(len(found_tweets), len(ids)))

        # Get the IDs for all Tweets
        for tweet in found_tweets:
            try:
                id = tweet.find_element_by_css_selector(id_selector).get_attribute('href').split('/')[-1]
                ids.append(id)
            except StaleElementReferenceException as e:
                print('lost element reference', tweet)

    except NoSuchElementException:
        print('no tweets on this day')

    start = increment_day(start, 1)

scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to load more tweets
scrolling down to lo

WebDriverException: Message: chrome not reachable
  (Session info: chrome=56.0.2924.87)
  (Driver info: chromedriver=2.28.455517 (2c6d2707d8ea850c862f04ac066724273981e88f),platform=Mac OS X 10.12.1 x86_64)


In [185]:
len(ids)

0

In [172]:
try:
    with open(twitter_ids_filename) as f:
        all_ids = ids + json.load(f)
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))
except FileNotFoundError:
    with open(twitter_ids_filename, 'w') as f:
        all_ids = ids
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))

with open(twitter_ids_filename, 'w') as outfile:
    json.dump(data_to_write, outfile)

print('Tweets Scraped!')
driver.close()

tweets found on this scrape:  364
total tweet count:  364
Tweets Scraped!


In [157]:
def store_tweet(tweet):
    # set table
    table = db['tweets']
    table.insert(tweet)

In [173]:
db_name = 'seattle'
db = dataset.connect('sqlite:///{}.sqlite'.format(db_name))

with open(twitter_ids_filename) as f:
    ids = json.load(f)
    for tweetId in ids:
        status = client.get_status(tweetId)
        
        if hasattr(status, 'entities'):
                hashtags = []
                for tag in status.entities['hashtags']:
                    hashtags.append(tag['text'])
                hashtags = json.dumps(hashtags)
                
                mentions = []
                for tag in status.entities['user_mentions']:
                    mentions.append(tag['screen_name'])
                mentions = json.dumps(mentions)
                
        tweet_dict = {
                    'loc': status.user.location,
                    'text': status.text,
                    'name': status.user.screen_name,
                    'followers': status.user.followers_count,
                    'id_str': status.id_str,
                    'retweet_count': status.retweet_count,
                    'friends_count': status.user.friends_count,
                    'favorite_count': status.favorite_count,
                    'time': status.created_at,
                    'hashtags': hashtags,
                    'mentions': mentions
                }
        store_tweet(tweet_dict)
        

In [177]:
import pandas as pd
import sqlite3

In [178]:
con = sqlite3.connect(db_name + ".sqlite")

In [179]:
df = pd.read_sql("SELECT * from tweets", con=con)

In [180]:
len(df)

364

In [181]:
df.head()

,id,followers,favorite_count,retweet_count,time,text,mentions,friends_count,hashtags,loc,name,id_str
0,1,8554,1,2,2016-01-14 16:19:49.000000,Here at #EFC44 listening to @MayorEdMurray tal...,"[""MayorEdMurray""]",2875,"[""EFC44"", ""earlyed""]","Seattle, Washington",weareoneamerica,687670462906671104
1,2,633,0,0,2015-11-02 23:46:08.000000,.@UnitedWayKC SHARE's response to @MayorEdMurr...,"[""UnitedWayKC"", ""MayorEdMurray""]",634,[],"Seattle, WA",share_shelters,661328470048247808
2,3,151,1,0,2016-10-13 00:17:14.000000,Shout out to @MayorEdMurray for sweeping ppl a...,"[""MayorEdMurray""]",552,"[""BloodsOnYourHands"", ""StopTheSweeps""]",Seattle,BLFsea,786360110285139969
3,4,4311,0,0,2016-12-09 03:33:56.000000,Shout out to @MayorEdMurray who just gave a co...,"[""MayorEdMurray""]",983,[],Washington,HassanPRG,807065718386671620
4,5,127,0,0,2017-03-07 05:53:31.000000,George Scarola met with folks from the Field a...,"[""MayorEdMurray""]",597,[],"Seattle, WA",AshleyA_RC,838990977352126464


In [122]:
df[df.name == 'beastard49']

,id,followers,favorite_count,retweet_count,time,text,mentions,friends_count,hashtags,loc,name,id_str
796,797,34,0,0,2017-03-30 15:32:58.000000,@cityofseattle @seattleoh how many properties ...,"[""CityofSeattle"", ""seattleoh""]",190,[],washington,beastard49,847471722391494657
